In [1]:
import json
import os
import csv
import urllib
from io import BytesIO
from PIL import Image

from socket import timeout

from google.colab import files

!pip3 install -q torch torchvision
import torch
from torchvision import models
from torch.utils.data import Dataset, SubsetRandomSampler, Sampler
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Download train data
!wget "https://storage.googleapis.com/dlcourse_ai/train.zip"
!unzip -q "train.zip"

train_folder = "train_kaggle/"
# Count number of files in the train folder, should be 4603
print('Number of files in the train folder', len(os.listdir(train_folder)))

# Download test data
!wget "https://storage.googleapis.com/dlcourse_ai/test.zip"
!unzip -q "test.zip"

test_folder = "test_kaggle/"
# Count number of files in the test folder, should be 1150
print('Number of files in the test folder', len(os.listdir(test_folder)))

--2022-12-16 14:45:41--  https://storage.googleapis.com/dlcourse_ai/train.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 74.125.24.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 562348083 (536M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>] 536.30M  22.1MB/s    in 25s     

2022-12-16 14:46:08 (21.4 MB/s) - ‘train.zip’ saved [562348083/562348083]

Number of files in the train folder 4603
--2022-12-16 14:46:12--  https://storage.googleapis.com/dlcourse_ai/test.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 142.251.10.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140788786 (134M) [application/zip]
Saving to: ‘test.zip’

test.zip 

In [3]:
device = torch.device("cuda:0")

In [4]:
class HotdogOrNotDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.transform = transform
        self.folder = folder
        self.filenames = os.listdir(folder)

    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, index):        
        # TODO Implement getting item by index
        # Hint: os.path.join is helpful!
        y = 0
        img_id = self.filenames[index]
        img = Image.open(os.path.join(self.folder, img_id))
        if self.transform:
            img = self.transform(img)
        if img_id.startswith("frankfurter") or img_id.startswith("chili-dog") or img_id.startswith("hotdog"):
            y = 1
        return img, y, img_id

In [5]:
# First, lets load the dataset
train_dataset = HotdogOrNotDataset("train_kaggle",
                       transform=transforms.Compose([
                           transforms.Resize((224, 224)),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
                          transforms.RandomHorizontalFlip(),
                          transforms.RandomAutocontrast()                       
                       ])
                      )

In [8]:
batch_size = 64

data_size = len(train_dataset)
validation_fraction = .2


val_split = int(np.floor((validation_fraction) * data_size))
indices = list(range(data_size))
np.random.seed(42)
np.random.shuffle(indices)

val_indices, train_indices = indices[:val_split], indices[val_split:]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                         sampler=val_sampler)

In [10]:
def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs, scheduler = None):
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        if scheduler is not None:
            scheduler.step()
        model.train() # Enter train mode

        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y, _) in enumerate(train_loader):
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            prediction = model(x_gpu)
            loss_value = loss(prediction, y_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()

            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y_gpu.shape[0]

            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)

        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        print("epoch %d Average loss: %f, Train accuracy: %f, val accuracy: %f" %
                  (epoch+1, ave_loss, train_accuracy, val_accuracy))
        if train_accuracy < 0.8:
            return loss_history, train_history, val_history

    return loss_history, train_history, val_history

def compute_accuracy(model, loader):
    """
    Computes accuracy on the dataset wrapped in a loader

    Returns: accuracy as a float value between 0 and 1
    """
    model.eval()
    correct_samples = 0
    total_samples = 0
    for i_step, (x, y, _) in enumerate(loader):
        x_gpu = x.to(device)
        y_gpu = y.to(device)
        prediction = model(x_gpu)
        _, indices = torch.max(prediction, 1)
        correct_samples += torch.sum(indices == y_gpu)
        total_samples += y_gpu.shape[0]

    train_accuracy = float(correct_samples) / total_samples
    return train_accuracy

In [12]:
from collections import namedtuple
Hyperparams = namedtuple("Hyperparams", ['lr', 'weight_decay', 'gamma', 'step_size'])
RunResult = namedtuple("RunResult", ['val_history', 'final_val_accuracy'])
run_record = {}

In [21]:
import torch.nn as nn
import torch.optim as optim

for i in range(10):
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)
    parameters = model.parameters()

    lr = 10**np.random.uniform(-5, -3)
    weight_decay = 10**np.random.uniform(-7, -2)
    gamma = np.random.uniform(0.01, 0.4)
    step_size = 4
    print(f"exp {i}", "lr =", lr, "(gamma, step_size) =", f"({gamma},{step_size})", "weight_decay =", weight_decay)

    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(parameters, lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    loss_history, train_history, val_history = train_model(model, train_loader, val_loader, loss, optimizer, 5, scheduler)
    hp = Hyperparams(lr=lr, weight_decay = weight_decay, gamma=gamma, step_size = step_size)
    rr = RunResult(val_history=val_history, final_val_accuracy=val_history[-1])
    run_record[hp] = rr

epoch 1 Average loss: 0.383189, Train accuracy: 0.834374, val accuracy: 0.927174
epoch 2 Average loss: 0.192848, Train accuracy: 0.934564, val accuracy: 0.940217
epoch 3 Average loss: 0.129341, Train accuracy: 0.957372, val accuracy: 0.954348
epoch 4 Average loss: 0.098522, Train accuracy: 0.970405, val accuracy: 0.960870
epoch 5 Average loss: 0.092499, Train accuracy: 0.972034, val accuracy: 0.951087
exp 1 lr = 1.2846401670472579e-05 (gamma, step_size) = (0.3899565851164549,4) weight_decay = 0.00016644993053753927
epoch 1 Average loss: 0.393942, Train accuracy: 0.838175, val accuracy: 0.909783
epoch 2 Average loss: 0.194745, Train accuracy: 0.939452, val accuracy: 0.938043
epoch 3 Average loss: 0.147225, Train accuracy: 0.951670, val accuracy: 0.954348
epoch 4 Average loss: 0.116022, Train accuracy: 0.961988, val accuracy: 0.941304
epoch 5 Average loss: 0.104817, Train accuracy: 0.966332, val accuracy: 0.948913
exp 2 lr = 1.3791176265215666e-05 (gamma, step_size) = (0.3016397964061552

In [22]:
result = sorted(list(run_record.items()), key=lambda x: x[1].final_val_accuracy)[::-1]
for a, b in result:
    print(a, b.final_val_accuracy)

Hyperparams(lr=0.00014757051437283905, weight_decay=0.0014755697881430009, gamma=0.2606052680262173, step_size=4) 0.966304347826087
Hyperparams(lr=8.123531813602789e-05, weight_decay=0.0002682271373776154, gamma=0.36498308359842513, step_size=4) 0.9652173913043478
Hyperparams(lr=0.0004173661021355521, weight_decay=1.1725660795879536e-05, gamma=0.08962495807561477, step_size=4) 0.9608695652173913
Hyperparams(lr=0.00018969868389929093, weight_decay=0.005857003576951157, gamma=0.3538300995524191, step_size=4) 0.9565217391304348
Hyperparams(lr=0.0003284812869193094, weight_decay=3.9847548290583244e-05, gamma=0.37111623523091136, step_size=4) 0.9565217391304348
Hyperparams(lr=0.00017733133063054424, weight_decay=1.2429609308571704e-07, gamma=0.10685991276607926, step_size=4) 0.9565217391304348
Hyperparams(lr=1.3791176265215666e-05, weight_decay=5.9352436069761535e-05, gamma=0.30163979640615524, step_size=4) 0.9554347826086956
Hyperparams(lr=0.0004207723628995787, weight_decay=1.208345982431

In [20]:
x = []
for a, b in result[:6]:
  x.append(a.gamma)
sorted(x)

[0.003207696512618674,
 0.10685991276607926,
 0.13208889858228565,
 0.37111623523091136,
 0.3829933438750959,
 0.4250854479572248]

In [23]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)
parameters = model.parameters()

lr = 0.00014757051437283905
weight_decay = 0.0014755697881430009
gamma = 0.2606052680262173
step_size = 4

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(parameters, lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

loss_history, train_history, val_history = train_model(model, train_loader, val_loader, loss, optimizer, 4, scheduler)

epoch 1 Average loss: 0.237348, Train accuracy: 0.901439, val accuracy: 0.945652
epoch 2 Average loss: 0.091412, Train accuracy: 0.967689, val accuracy: 0.886957
epoch 3 Average loss: 0.079242, Train accuracy: 0.968504, val accuracy: 0.945652
epoch 4 Average loss: 0.035452, Train accuracy: 0.986696, val accuracy: 0.963043
epoch 5 Average loss: 0.019188, Train accuracy: 0.995656, val accuracy: 0.957609
epoch 6 Average loss: 0.011454, Train accuracy: 0.997013, val accuracy: 0.957609
epoch 7 Average loss: 0.007588, Train accuracy: 0.998099, val accuracy: 0.959783
epoch 8 Average loss: 0.007241, Train accuracy: 0.998914, val accuracy: 0.960870
epoch 9 Average loss: 0.007703, Train accuracy: 0.998371, val accuracy: 0.964130
epoch 10 Average loss: 0.005236, Train accuracy: 0.999728, val accuracy: 0.961957
epoch 11 Average loss: 0.003747, Train accuracy: 0.999728, val accuracy: 0.957609
epoch 12 Average loss: 0.004214, Train accuracy: 0.999728, val accuracy: 0.960870
epoch 13 Average loss: 0.

KeyboardInterrupt: ignored

In [24]:
class SubsetSampler(Sampler):
    r"""Samples elements with given indices sequentially

    Arguments:
        indices (ndarray): indices of the samples to take
    """

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)
    
    
def evaluate_model(model, dataset, indices):
    """
    Computes predictions and ground truth labels for the indices of the dataset
    
    Returns: 
    predictions: np array of ints - model predictions
    grount_truth: np array of ints - actual labels of the dataset
    """
    model.eval()
    predictions = []
    ground_truth = []
    sampler = SubsetSampler(indices)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                           sampler=sampler)
    for i_step, (x, y, _) in enumerate(loader):
        prediction = model(x)
        _, ind = torch.max(prediction, 1)
        predictions.extend(ind)
        ground_truth.extend(y)

    return predictions, ground_truth

model.to('cpu')
predictions, gt = evaluate_model(model, train_dataset, val_indices)

def binary_classification_metrics(prediction, ground_truth):
    if type(prediction) == type(list()):
      prediction = np.array(prediction)
    if type(ground_truth) == type(list()):
      ground_truth = np.array(ground_truth)
    precision = np.sum(prediction & ground_truth) / np.sum(prediction)
    recall = np.sum(prediction & ground_truth) / np.sum(ground_truth)
    f1 = 2 / (1/precision + 1/recall)

    return precision, recall, f1


precision, recall, f1 = binary_classification_metrics(predictions, gt)
print("F1: %4.3f, P: %4.3f, R: %4.3f" % (f1, precision, recall))

F1: 0.937, P: 0.945, R: 0.929
